# notes: bounding box expander utility (for LabelMe JSON format files)

In [ ]:
import os
import json
import numpy as np
import glob
import argparse
from pathlib import Path
from tqdm import tqdm

def expand_bounding_box(points, expand_ratio, image_width, image_height, shape_type):
    """
    Expand a bounding box by a given ratio while keeping it within image boundaries.
    
    Args:
        points (list): Points defining the bounding box
        expand_ratio (float): Expansion ratio (e.g., 0.5 for 50% increase)
        image_width (int): Width of the image in pixels
        image_height (int): Height of the image in pixels
        shape_type (str): Type of shape ('rectangle' or 'polygon')
        
    Returns:
        list: Expanded points
    """
    # Convert points to numpy array for easier manipulation
    points_array = np.array(points)
    
    if shape_type == "rectangle":
        # Rectangle in labelme has [x1,y1], [x2,y2] format (top-left and bottom-right corners)
        x1, y1 = points_array[0]
        x2, y2 = points_array[1]
        
        # Calculate center and current dimensions
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = abs(x2 - x1)
        height = abs(y2 - y1)
        
        # Calculate new dimensions with expansion
        new_width = width * (1 + expand_ratio)
        new_height = height * (1 + expand_ratio)
        
        # Calculate new corners while keeping within image boundaries
        new_x1 = max(0, center_x - new_width / 2)
        new_y1 = max(0, center_y - new_height / 2)
        new_x2 = min(image_width, center_x + new_width / 2)
        new_y2 = min(image_height, center_y + new_height / 2)
        
        # Return new points
        return [[new_x1, new_y1], [new_x2, new_y2]]
    
    elif shape_type == "polygon":
        # For polygons, we'll expand from the centroid
        # Calculate centroid (center) of the polygon
        centroid_x = np.mean(points_array[:, 0])
        centroid_y = np.mean(points_array[:, 1])
        
        # Create expanded points by moving each point away from centroid
        expanded_points = []
        for point in points_array:
            # Vector from centroid to point
            vector_x = point[0] - centroid_x
            vector_y = point[1] - centroid_y
            
            # Expand the vector
            expanded_x = centroid_x + vector_x * (1 + expand_ratio)
            expanded_y = centroid_y + vector_y * (1 + expand_ratio)
            
            # Ensure the point stays within image boundaries
            expanded_x = max(0, min(expanded_x, image_width))
            expanded_y = max(0, min(expanded_y, image_height))
            
            expanded_points.append([expanded_x, expanded_y])
        
        return expanded_points
    
    else:
        # For unsupported shape types, return original points
        print(f"Warning: Shape type '{shape_type}' not supported for expansion. Keeping original.")
        return points

def process_labelme_json(file_path, output_dir, expand_ratio=0.5):
    """
    Process a labelme JSON file to expand all bounding boxes.
    
    Args:
        file_path (str): Path to the labelme JSON file
        output_dir (str): Directory to save the processed JSON file
        expand_ratio (float): Ratio by which to expand the bounding boxes
        
    Returns:
        str: Path to the processed JSON file
    """
    # Load the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Get image dimensions
    image_height = data["imageHeight"]
    image_width = data["imageWidth"]
    
    # Process each shape
    for shape in data["shapes"]:
        shape_type = shape["shape_type"]
        # Expand the bounding box
        shape["points"] = expand_bounding_box(
            shape["points"], 
            expand_ratio, 
            image_width, 
            image_height, 
            shape_type
        )
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the processed JSON file
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)
    
    return output_file

def main(input_dir, output_dir, expand_ratio=0.5, file_pattern="*.json"):
    """
    Process all labelme JSON files in a directory to expand bounding boxes.
    
    Args:
        input_dir (str): Directory containing labelme JSON files
        output_dir (str): Directory to save processed JSON files
        expand_ratio (float): Ratio by which to expand the bounding boxes
        file_pattern (str): Pattern to match JSON files
    """
    # Find all JSON files in the input directory
    json_files = glob.glob(os.path.join(input_dir, file_pattern))
    print(f"Found {len(json_files)} JSON files in {input_dir}")
    
    # Process each JSON file
    for json_file in tqdm(json_files, desc="Processing JSON files"):
        output_file = process_labelme_json(json_file, output_dir, expand_ratio)
        #print(f"Processed {os.path.basename(json_file)} -> {os.path.basename(output_file)}")
    
    print(f"All files processed and saved to {output_dir}")

# Example usage in Jupyter Notebook
if __name__ == "__main__":
    # Example parameters
    # default : '/path/to/your/labelme/annotations' ;
    # alt : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\temp-0\\samples'
    # example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\HM_aligned_to_SC_renamed_files_manual_labels_T_(new_OM_images)'
    # example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\HM_aligned_to_SC_renamed_files_manual_labels_S_(new_OM_images)'
    # example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\HM_aligned_to_SC_renamed_files_automated_labels_T'
    # example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\HM_aligned_to_SC_renamed_files_automated_labels_S'
    INPUT_DIR = "C:\\Users\\praam\\Desktop\\havetai+vetcyto\\HM_aligned_to_SC_renamed_files_manual_labels_T_(new_OM_images)"
    # default : '/path/to/your/expanded/annotations' ;
    # alt 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\temp-1\\samples\\by_50_percent'
    # alt 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\temp-1\\samples\\by_25_percent'
    # example 01 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\50_pc_expanded_manual_labels_with_aligned_HM_images_T'
    # example 02 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\25_pc_expanded_manual_labels_with_aligned_HM_images_T'
    # example 03 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\50_pc_expanded_automated_labels_with_aligned_HM_images_T'
    # example 04 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\25_pc_expanded_automated_labels_with_aligned_HM_images_T'
    # example 05 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\50_pc_expanded_manual_labels_with_aligned_HM_images_S'
    # example 06 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\25_pc_expanded_manual_labels_with_aligned_HM_images_S'
    # example 07 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\50_pc_expanded_automated_labels_with_aligned_HM_images_S'
    # example 08 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\25_pc_expanded_automated_labels_with_aligned_HM_images_S'
    # example 09 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\75_pc_expanded_manual_labels_with_aligned_HM_images_T'
    # example 10 : 'C:\\Users\\praam\\Desktop\\havetai+vetcyto\\75_pc_expanded_automated_labels_with_aligned_HM_images_T'
    OUTPUT_DIR = "C:\\Users\\praam\\Desktop\\havetai+vetcyto\\75_pc_expanded_automated_labels_with_aligned_HM_images_T"
    EXPAND_RATIO = 0.75  # either 0.5 for 50% expansion or 0.25 for 25% expansion & maybe 0.75 for 75% expansion 
    
    # Call the main function
    main(INPUT_DIR, OUTPUT_DIR, EXPAND_RATIO)

# Notes: The end.